In [ ]:
import pandas as pd
import re
from collections import Counter
import numpy as np

In [ ]:
firstg_df = pd.read_csv(
    "Google Play Store Data/google_data_first.csv", encoding="ISO-8859-1"
)

#### Group categories

In [ ]:
app_categories = {
    "Games": [
        "Racing",
        "Puzzle",
        "Arcade",
        "Role Playing",
        "Card",
        "Casual",
        "Action",
        "Board",
        "Casino",
        "Trivia",
        "Strategy",
        "Adventure",
        "Word",
        "Brain Games",
        "Pretend Play",
        "Action & Adventure",
    ],
    "Entertainment": [
        "Photography",
        "Entertainment",
        "Music & Video",
        "Music & Audio",
        "Music",
        "Art & Design",
        "Creativity",
        "Simulation",
        "Sports",
        "Comics",
    ],
    "Utilities": [
        "Personalization",
        "Tools",
        "Books & Reference",
        "Social",
        "News & Magazines",
        "Maps & Navigation",
        "Finance",
        "Productivity",
        "Education",
        "Medical",
        "Communication",
        "Business",
        "Video Players & Editors",
        "Travel & Local",
        "Lifestyle",
        "Health & Fitness",
        "Auto & Vehicles",
        "Weather",
        "Libraries & Demo",
        "Food & Drink",
        "House & Home",
        "Events",
        "Parenting",
        "Beauty",
        "Dating",
        "Shopping",
        "Educational",
    ],
}
hedonic_categories = {
    "Hedonic": [
        "Racing",
        "Puzzle",
        "Arcade",
        "Role Playing",
        "Card",
        "Casual",
        "Action",
        "Board",
        "Casino",
        "Trivia",
        "Strategy",
        "Adventure",
        "Word",
        "Brain Games",
        "Pretend Play",
        "Action & Adventure",
        "Photography",
        "Entertainment",
        "Music & Video",
        "Music & Audio",
        "Music",
        "Art & Design",
        "Creativity",
        "Simulation",
        "Sports",
        "Comics",
        "Dating",
        "Social",
    ],
    "Utilitarian": [
        "Personalization",
        "Tools",
        "Books & Reference",
        "News & Magazines",
        "Maps & Navigation",
        "Finance",
        "Productivity",
        "Education",
        "Medical",
        "Communication",
        "Business",
        "Video Players & Editors",
        "Travel & Local",
        "Lifestyle",
        "Health & Fitness",
        "Auto & Vehicles",
        "Weather",
        "Libraries & Demo",
        "Food & Drink",
        "House & Home",
        "Events",
        "Parenting",
        "Beauty",
        "Shopping",
        "Educational",
    ],
}

### Pre-process

In [ ]:
## Data types columns first:

# my_app_id = string
# date_published = datetime
# privacy_policy = convert to boolean "privacy_policy"
# rating_app = float
# nb_rating = integer
# num_downloads = integer - ranges
# content_rating_app = split(pegi, pegi_reason)
# developer = string, unique
# categ_app = string
# (in_app = boolean)
# (has_ads = boolean)
# price_gplay = convert to pounds, float
# operating_system = range, probably drop
# software_version = string_number
# price_gplay = convert euro to pound;
# interactive_element = too many empty rows, use sdk if necessary
# in_app_product = ranges and single prices; convert euros
# developer_name = string, more nan than developer; drop
# nb_screenshots = integer
# description = string
# whats_new = empty, drop
# email_to = convert to boolean "email"
# developer_info = string, drop
# similar_apps_top15 = never 15 apps, convert to list; split on komma
# visit_website = convert to boolean "website"
# more_from_developer = convert to list, split on komma; 1/4 nan
# (family_library = boolean)
# permissions = list of strings

In [ ]:
def find_special_freemium(df: pd.DataFrame, drop_lst):
    def clean_app_name(name):
        return re.sub(r"demo|trial|free|lite|basic|sampler|preview", "", name.lower())

    def find_paid_version(row):
        base_name = clean_app_name(row["my_app_id"])
        return base_name if base_name in prem_lst else None

    def replace_rating(merged_df):
        if pd.isnull(merged_df["rating_app_free"]) and not pd.isnull(
            merged_df["rating_app_paid"]
        ):
            return merged_df["rating_app_paid"], (
                merged_df["nb_rating_free"] + merged_df["nb_rating_paid"]
            )

        if not pd.isnull(merged_df["rating_app_free"]) and pd.isnull(
            merged_df["rating_app_paid"]
        ):
            return merged_df["rating_app_free"], (
                merged_df["nb_rating_free"] + merged_df["nb_rating_paid"]
            )
        try:
            return (
                (merged_df["nb_rating_free"] * merged_df["rating_app_free"])
                + (merged_df["nb_rating_paid"] * merged_df["rating_app_paid"])
            ) / (merged_df["nb_rating_free"] + merged_df["nb_rating_paid"]), (
                merged_df["nb_rating_free"] + merged_df["nb_rating_paid"]
            )
        except:
            return np.NaN, 0

    prem_lst = set(df[df["premium"] == 1]["my_app_id"].apply(clean_app_name))
    freetrial_df = df[
        (df["price_gplay"] == 0)
        & df["my_app_id"].str.contains(
            "trial|demo|free|lite|basic|sampler|preview", case=False, na=False
        )
    ]

    freetrial_df["paidv_app_id"] = freetrial_df.apply(find_paid_version, axis=1)

    prem_df = df[df["premium"] == 1].copy()
    prem_df["my_app_id"] = prem_df["my_app_id"].apply(clean_app_name)
    prem_df["my_app_id"] = prem_df["my_app_id"].str.lower()
    merged_df = pd.merge(
        freetrial_df,
        prem_df,
        how="inner",
        left_on="paidv_app_id",
        right_on="my_app_id",
        suffixes=("_free", "_paid"),
    )
    merged_df.drop(drop_lst, inplace=True)

    rating_combination = merged_df.apply(replace_rating, axis=1)
    merged_df["combined_rating"] = [x[0] for x in rating_combination]
    merged_df["combined_nb_rating"] = [x[1] for x in rating_combination]

    filter_df = merged_df[merged_df.columns.drop(list(merged_df.filter(regex="paid")))]
    filter_df.columns = filter_df.columns.str.replace("_free", "", regex=False)
    filter_df["freemium"] = 1
    filter_df["premium"] = 0
    filter_df["label"] = 0
    filter_df["nb_rating"] = filter_df["combined_nb_rating"]
    filter_df["rating_app"] = filter_df["combined_rating"]
    filter_df.drop(["combined_nb_rating", "combined_rating"], axis=1, inplace=True)

    df = (
        pd.concat([df, filter_df])
        .drop_duplicates(subset=["my_app_id"], keep="last")
        .reset_index(drop=True)
    )

    return df

In [ ]:
def preprocess_gps(df: pd.DataFrame, app_categories):
    """Function to preprocess the googleplaystore data.
    It performs a few different transformations, mainly
    related to string extraction.

    Input: df[pd.DataFrame]
    Output: df[pd.DataFrame]"""

    df = df.copy()

    # drop unnecessaty columns
    df = df.drop(
        columns=[
            "Unnamed: 0",
            "developer_name",
            "whats_new",
            "developer_info",
            "in_app_product",
        ]
    )

    # transform to publish date to datetime object
    df["date_published"] = pd.to_datetime(
        df["date_published"].str.replace("</span", "")
    )

    # extract the number of days since release
    df["days_released"] = (df["date_published"].max() - df["date_published"]).dt.days

    # create the boolean variables for features on contact
    bool_cols = ["privacy_policy", "email_to", "visit_website"]
    df[bool_cols] = df[bool_cols].fillna(value=0).apply(lambda x: x != 0).astype(bool)
    df.rename(
        columns={
            "email_to": "email",
            "similar_apps_top15": "similar_apps_top5",
        },
        inplace=True,
    )

    # transform the numeric variables
    df["nb_rating"] = df["nb_rating"].fillna("0").str.replace(",", "").astype(int)
    df["nb_screenshots"] = df["nb_screenshots"].fillna(0).astype(int)
    df["software_version"] = df["software_version"].str.extract(r"(\d+(\.\d+)*)")[0]

    # extract the pegi rating and reason
    df["pegi_rating"] = (
        df["content_rating_app"].str.extract(r"PEGI (\d+)").fillna(0).astype(int)
    )
    df["pegi_reason"] = (
        df["content_rating_app"]
        .str.split(" ", n=2)
        .str[2]
        .fillna("Unknown")
        .apply(lambda x: x.strip())
        .str.split(",")
        .tolist()
    )

    df["pegi_category"] = pd.Categorical(
        df["pegi_rating"], categories=sorted(df["pegi_rating"].unique()), ordered=True
    ).codes

    # create lists for the following columns
    cols = [
        "similar_apps_top5",
        "more_from_developer",
        "permissions",
        "interactive_element",
    ]
    for i in cols:
        df[i] = (
            df[i]
            .fillna("Unknown")
            .str.split(",")
            .apply(lambda x: [item.strip() for item in x])
        )

    # Custom function to count features, treating "Unknown" as 0
    def count_length(feature_list):
        return len([feature for feature in feature_list if feature != "Unknown"])

    # create new features
    df["nb_similar_apps"] = df["similar_apps_top5"].apply(count_length)
    df["nb_permissions"] = df["permissions"].apply(count_length)
    df["nb_interactive_el"] = df["interactive_element"].apply(count_length)

    # create nb_features
    df["nb_features"] = df["nb_permissions"] + df["nb_interactive_el"]

    # transform the num_download into more insightful columns
    df["min_downloads"] = (
        df["num_downloads"]
        .fillna("0")
        .str.replace(",", "")
        .str.extract(r"(\d+(\.\d+)*)")[0]
        .astype("int64")
    )

    df["intv_downloads"] = pd.cut(
        df["min_downloads"], bins=sorted(list(set(df["min_downloads"]))), right=False
    )

    # Function to calculate indegree for each app
    def calculate_indegree(app_list, app_counts):
        return sum(app_counts[app] for app in app_list)

    # Flatten the 'similarity_apps_top5' column and count app occurrences
    app_counts = Counter(app for sublist in df["similar_apps_top5"] for app in sublist)

    # Apply the function to the 'similarity_apps_top5' column using .apply()
    df["nb_recommendations"] = df["similar_apps_top5"].apply(
        lambda x: calculate_indegree(x, app_counts)
    )

    # Function to map specific categories to general categories
    def map_categories(categ, app_dict):
        for new_categ, categs in app_dict.items():
            if categ in categs:
                return new_categ
        return None

    # Apply the mapping function to the 'category' column
    df["hedonic"] = df["categ_app"].apply(
        lambda x: map_categories(categ=x, app_dict=app_categories)
    )

    # Function to convert price from pounds to euros based on the conversion rate on January 1, 2018
    def convert_pounds_to_euros(price_in_pounds):
        # Conversion rate as of January 1, 2018: 1 GBP = 1.1283 EUR
        try:
            price_in_pounds = float(price_in_pounds.replace("£", ""))
            price_in_euros = price_in_pounds * 1.1283
            return round(price_in_euros, 2)
        except:
            return None

    df["price_gplay"] = df["price_gplay"].apply(convert_pounds_to_euros)

    df.dropna(subset=["price_gplay", "categ_app", "operating_system"], inplace=True)

    df = df[df["operating_system"] != "Varies with device"]
    df["operating_system"] = pd.Categorical(
        df["operating_system"].apply(lambda x: x.split(" ")[0])
    ).codes

    df["nb_apps_dev"] = df.groupby("developer")["my_app_id"].transform("count")

    def create_label(row):
        if row["price_gplay"] != 0:
            return 1
        elif row["in_app"] and row["price_gplay"] == 0:
            return 0
        else:
            return 2

    # Create dummies for business model types
    df["label"] = df.apply(create_label, axis=1)
    dummy_df = pd.get_dummies(df["label"], prefix="label", dtype=int)
    df = pd.concat([df, dummy_df], axis=1).rename(
        columns={"label_0": "freemium", "label_1": "premium", "label_2": "free"}
    )

    # Find the length of the description
    df["nb_words"] = df["description"].apply(lambda x: len(x.split()))
    df["nb_letters"] = df["description"].apply(lambda x: len(x))

    df = find_special_freemium(df, drop_lst=[1225, 508, 509])
    df.dropna(subset="rating_app", inplace=True)

    print(df.columns)
    return df

In [ ]:
firstg_df = preprocess_gps(firstg_df, hedonic_categories)

C:\Users\20202016\AppData\Local\Temp\ipykernel_9136\1402006418.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  freetrial_df["paidv_app_id"] = freetrial_df.apply(find_paid_version, axis=1)
C:\Users\20202016\AppData\Local\Temp\ipykernel_9136\1402006418.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df["freemium"] = 1
C:\Users\20202016\AppData\Local\Temp\ipykernel_9136\1402006418.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

Index(['my_app_id', 'date_published', 'privacy_policy', 'rating_app',
       'nb_rating', 'num_downloads', 'content_rating_app', 'developer',
       'categ_app', 'in_app', 'has_ads', 'price_gplay', 'operating_system',
       'software_version', 'interactive_element', 'nb_screenshots',
       'description', 'email', 'similar_apps_top5', 'visit_website',
       'more_from_developer', 'family_library', 'permissions', 'days_released',
       'pegi_rating', 'pegi_reason', 'pegi_category', 'nb_similar_apps',
       'nb_permissions', 'nb_interactive_el', 'nb_features', 'min_downloads',
       'intv_downloads', 'nb_recommendations', 'hedonic', 'nb_apps_dev',
       'label', 'freemium', 'premium', 'free', 'nb_words', 'nb_letters'],
      dtype='object')


In [ ]:
firstg_df

,my_app_id,date_published,privacy_policy,rating_app,nb_rating,num_downloads,content_rating_app,developer,categ_app,in_app,...,intv_downloads,nb_recommendations,hedonic,nb_apps_dev,label,freemium,premium,free,nb_words,nb_letters
0,AOIC.AOIC_anb01,2012-09-13,False,3.000000,2,100+ >,PEGI 3 Learn More,aoi-lab,Photography,False,...,"[100, 500)",894,Hedonic,1,1,0,1,0,133,881
1,Adrenaline.Crew,2018-08-29,True,3.700000,155,"5,000+",PEGI 12 Moderate Violence,Adrenaline+Crew,Racing,True,...,"[5000, 10000)",2207,Hedonic,5,1,0,1,0,364,2056
2,Alekseyt.Lusher,2015-01-24,False,4.300000,7539,"1,000,000+",PEGI 3 Learn More,Alekseyt,Entertainment,False,...,"[1000000, 5000000)",1330,Hedonic,6,2,0,0,1,99,622
4,Alfasoft.Widgets.CroatiaFlagAnalogClock,2014-01-07,False,5.000000,1,10+,PEGI 3 Learn More,Alfasoft,Personalization,False,...,"[10, 50)",4366,Utilitarian,18,1,0,1,0,121,675
10,Alfasoft.Widgets.PsychedelicAnalogClock,2014-01-09,False,3.900000,416,"50,000+",PEGI 3 Learn More,Alfasoft,Personalization,False,...,"[50000, 100000)",4736,Utilitarian,18,2,0,0,1,120,674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728779,zh.wang.android.SunflowerLiveWallpaperFree,2013-06-07,False,4.166485,1098,"100,000+",PEGI 3 Learn More,Zhenghong+Wang,Personalization,False,...,"[100000, 500000)",685,Utilitarian,9,0,1,0,1,560,3413
728780,zheng.meizi.com.dataswitchfree,2017-12-23,False,4.408152,368,"10,000+",PEGI 3 Learn More,JTMZ+Group,Tools,False,...,"[10000, 50000)",4450,Utilitarian,12,0,1,0,1,21,116
728781,zhivova.lyubov.crosswordpuzzlesforkidslite,2016-05-04,True,4.236842,19,"10,000+",PEGI 3 Learn More,Lyubov+Zhivova,Brain Games,False,...,"[10000, 50000)",4438,Hedonic,6,0,1,0,1,103,615
728782,zhivova.lyubov.crucigramasparaninoslite,2016-05-04,True,4.091304,23,"5,000+",PEGI 3 Learn More,Lyubov+Zhivova,Brain Games,False,...,"[5000, 10000)",4611,Hedonic,6,0,1,0,1,91,558


In [ ]:
firstg_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 659629 entries, 0 to 728783
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   my_app_id            659629 non-null  object        
 1   date_published       659629 non-null  datetime64[ns]
 2   privacy_policy       659629 non-null  bool          
 3   rating_app           659629 non-null  float64       
 4   nb_rating            659629 non-null  int64         
 5   num_downloads        659629 non-null  object        
 6   content_rating_app   659629 non-null  object        
 7   developer            659629 non-null  object        
 8   categ_app            659629 non-null  object        
 9   in_app               659629 non-null  bool          
 10  has_ads              659629 non-null  bool          
 11  price_gplay          659629 non-null  float64       
 12  operating_system     659629 non-null  int8          
 13  software_version   

In [ ]:
firstg_df.to_csv("begin2018.csv")